**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:
!pip install scikit-video

     |████████████████████████████████| 2.3MB 4.7MB/s 


In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import matplotlib.pyplot as plt

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization
from keras.layers import *
from keras.optimizers import *

from keras.models import Model
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

(you can use Markdown and Latex)

The function can either be in trained mode, or not. 

If the agent is not in train mode and considers that is has fully learned the best policy, it will always choose the act given by said policy.

If the agent is still in train mode, it will alternate between exploration and exploitation, with a probability $\epsilon$. Hence, with a probability $\epsilon$, the agent will be in exploration mode and will choose a random action (with equiprobability). Otherwise, it will be in exploitation mode and whill choose the best action according to the policy that it has learned so far. 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = *Environment*()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
        
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3)) + 128
        b[self.board>0, 0] = 256 #red: cheese
        b[self.board < 0, 2] = 256 #blue: poison
        b[self.x,self.y,:] = 256 #black: cat
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b

    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T = 200
temperature = 0.3
epochs_train = 20 # set small when debugging
epochs_test = 20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.





*   ```position```: indicates where the cat can go, i.e. -1 on the border to make sure the cat does not go out of bounds, 1 where the cat actually is, and 0 everywhere else.
*   ```board```: indicates the reward associated to each cell, i.e. whether there is cheese, poison or nothing.



## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a = np.random.randint(0, self.n_action, size=1)[0] #select a random action
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
          # The agent performs an action
          action = agent.act(state)

          # Apply an action to the environment, get the next state, the reward
          # and if the games end
          state, reward, game_over = env.act(action)

          # Update the counters
          if reward > 0:
                win = win + reward
          if reward < 0:
                lose = lose - reward

        # Save as a mp4
        env.draw(prefix + str(e))

        # Update stats
        score = score + win - lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 12.0/11.0. Average score (1.0)
Win/lose count 8.0/5.0. Average score (2.0)
Win/lose count 5.0/8.0. Average score (0.3333333333333333)
Win/lose count 13.0/16.0. Average score (-0.5)
Win/lose count 10.0/20.0. Average score (-2.4)
Win/lose count 9.5/18.0. Average score (-3.4166666666666665)
Win/lose count 9.5/16.0. Average score (-3.857142857142857)
Win/lose count 13.5/5.0. Average score (-2.3125)
Win/lose count 10.5/17.0. Average score (-2.7777777777777777)
Win/lose count 5.5/17.0. Average score (-3.65)
Win/lose count 11.5/9.0. Average score (-3.090909090909091)
Win/lose count 6.0/9.0. Average score (-3.0833333333333335)
Win/lose count 6.5/18.0. Average score (-3.730769230769231)
Win/lose count 7.5/14.0. Average score (-3.9285714285714284)
Win/lose count 8.5/10.0. Average score (-3.7666666666666666)
Win/lose count 12.5/15.0. Average score (-3.6875)
Win/lose count 9.5/9.0. Average score (-3.4411764705882355)
Win/lose count 8.5/11.0. Average score (-3.388888888888889)
Win/lo

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




*Solution, part 1:*

\begin{equation*}
\begin{aligned}
Q^{\pi}(s,a)=& E_{p^{\pi}}[\sum_{t\geq 0}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
=& r(s,a) + E_{p^{\pi}}[\sum_{t\geq 1}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
=& r(s,a) + \gamma E_{p^{\pi}}[ E_{p^{\pi}}[\sum_{t\geq 1}\gamma^{t-1}r(s_{t},a_{t})|s_{1}=s',a_{1}=a']|s_{0}=s,a_{0}=a]\\
=& r(s,a) + \gamma \sum_{(s',a')} p(s_{t+1} = s', a_{t+1} = a' |s_{t} = s', a_{t} = a') E_{p^{\pi}}[\sum_{t\geq 1}\gamma^{t-1}r(s_{t},a_{t})|s_{1}=s',a_{1}=a']\\
=& \sum_{(s',a')} p(s_{t+1} = s', a_{t+1} = a' |s_{t} = s', a_{t} = a')r(s,a) + \gamma \sum_{(s',a')} p(s_{t+1} = s', a_{t+1} = a' |s_{t} = s', a_{t} = a') E_{p^{\pi}}[\sum_{t\geq 1}\gamma^{t-1}r(s_{t},a_{t})|s_{1}=s',a_{1}=a']\\
=& E_{(s',a')\sim p(.|s,a)}[r(s,a) + E_{p^{\pi}}[\sum_{t\geq 0}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s',a_{0}=a']\\
=&E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{aligned}
\end{equation*}

*Solution, part 2:*

\begin{equation*}
\begin{aligned}
Q^*(s,a)=& max_{\pi}Q^{\pi}(s,a) \\
=&Q^{\pi^*}(s,a) \\
=&E_{(s',a')\sim p^*(.|s,a)}[r(s,a)+\gamma Q^{\pi^*}(s',a')]\\
=&E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma \max_{a'}Q^*(s',a')] \text{ since the optimal policy is deterministic and selects actions with } \pi^*(s) = argmax_(a) Q^*(s,a)
\end{aligned}
\end{equation*}

*Solution, part 3:*
The previous equations shows that the optimal Q-value is the unique fixed point of the Bellman operator. Therefore, choosing this loss entails reaching a Q-function that minimizes the Bellman error. Hence, if the algorithm converges, it will converge to the optimal Q-value.



***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) <= self.max_memory:
            self.memory.append(m) # add to memory
        else:
            del self.memory[0]  # if memory is saturated, we remove the first element
            
    def random_access(self, size=1):
        return self.memory[np.random.randint(len(self.memory))] # return random element

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s[None])) # most likely action

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        # recall that state = 2-neighborhood of mouse in the grid
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))  # initialize states
        target_q = np.zeros((self.batch_size, 4))  # initialize targets
        
        for i in range(self.batch_size):
            # draw a sample
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            # bootstrap a target value
            input_states[i] = s_
            target_q[i] = self.model.predict(np.array([s_]))[0]

            if game_over_:
                # in this case only the reward counts because the episode terminates
                target_q[i, a_] = r_
            else:
                target_q[i, a_] = r_ + self.discount * self.model.predict(n_s_.reshape(1, 5, 5, self.n_state)).max()
            
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__(*args, **kwargs)
        
        # NN Model
        input_shape = (5, 5, self.n_state)  # give the 2-neighborhood of cat
        hid_size1 = 64
        hid_size2 = 4
        
        inp = Input(shape=input_shape)
        x = Flatten()(inp)
        x = Dense(hid_size1, activation="relu")(x)
        x = Dense(hid_size2, activation="linear")(x)
        model = Model(inputs=inp, outputs=x)
        
        model.compile(adam(lr=lr, decay=1e-4), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=1e-3, epsilon=0.1, memory_size=2000, batch_size=128)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/020 | Loss 0.0077 | Win/lose count 2.5/10.0 (-7.5)
Epoch 001/020 | Loss 0.0170 | Win/lose count 2.0/4.0 (-2.0)
Epoch 002/020 | Loss 0.0102 | Win/lose count 1.0/5.0 (-4.0)
Epoch 003/020 | Loss 0.0144 | Win/lose count 2.5/1.0 (1.5)
Epoch 004/020 | Loss 0.0285 | Win/lose count 3.0/9.0 (-6.0)
Epoch 005/020 | Loss 0.0036 | Win/lose count 6.5/4.0 (2.5)
Epoch 006/020 | Loss 0.0208 | Win/lose count 2.0/2.0 (0.0)
Epoch 007/020 | Loss 0.0017 | Win/lose count 1.0/1.0 (0.0)
Epoch 008/020 | Loss 0.0047 | Win/lose count 1.5/0 (1.5)
Epoch 009/020 | Loss 0.0155 | Win/lose count 6.0/2.0 (4.0)
Epoch 010/020 | Loss 0.0015 | Win/lose count 3.5/3.0 (0.5)
Epoch 011/020 | Loss 0.0201 | Win/lose count 10.5/2.0 (8.5)
Epoch 012/020 | Loss 0.0011 | Win/lose count 2.5/1.0 (1.5)
Epoch 013/020 | Loss 0.0009 | Win/lose count 2.5/0 (2.5)
Epoch 014/020 | Loss 0.0032 | Win/lose count 3.5/4.0 (-0.5)
Epoch 015/020 | Loss 0.0171 | Win/lose count 6.0/3.0 (3.0)
Epoch 016/020 | Loss 0.0087 | Win/lose count 5.0/3.0 

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args, lr=0.1, **kwargs):
        super(DQN_CNN, self).__init__(*args, **kwargs)

        input_shape = (5, 5, self.n_state)  # give the 2-neighborhood of cat
      
        inp = Input(shape=input_shape)
        x = Conv2D(16, (3, 3), activation='relu')(inp)
        x = Conv2D(16, (3, 3), activation='relu')(x)
        x = Flatten()(x)
        x = Dense(4, activation="linear")(x) 

        model = Model(inputs=inp, outputs=x)
        model.compile(adam(lr=lr, decay=1e-4), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=1e-3, epsilon = 0.1, memory_size=2000, batch_size=32)
train(agent, env, epochs_train, prefix='cnn_train')
HTML(display_videos(f'cnn_train10.mp4'))

Epoch 000/020 | Loss 0.0018 | Win/lose count 4.0/9.0 (-5.0)
Epoch 001/020 | Loss 0.0049 | Win/lose count 3.5/4.0 (-0.5)
Epoch 002/020 | Loss 0.0019 | Win/lose count 4.0/2.0 (2.0)
Epoch 003/020 | Loss 0.0607 | Win/lose count 2.5/5.0 (-2.5)
Epoch 004/020 | Loss 0.0015 | Win/lose count 4.0/1.0 (3.0)
Epoch 005/020 | Loss 0.0022 | Win/lose count 4.5/3.0 (1.5)
Epoch 006/020 | Loss 0.0049 | Win/lose count 5.0/3.0 (2.0)
Epoch 007/020 | Loss 0.0554 | Win/lose count 6.0/5.0 (1.0)
Epoch 008/020 | Loss 0.0028 | Win/lose count 4.0/4.0 (0.0)
Epoch 009/020 | Loss 0.0027 | Win/lose count 12.0/1.0 (11.0)
Epoch 010/020 | Loss 0.0227 | Win/lose count 10.5/2.0 (8.5)
Epoch 011/020 | Loss 0.0017 | Win/lose count 9.0/3.0 (6.0)
Epoch 012/020 | Loss 0.0017 | Win/lose count 10.0/3.0 (7.0)
Epoch 013/020 | Loss 0.0036 | Win/lose count 5.5/2.0 (3.5)
Epoch 014/020 | Loss 0.0030 | Win/lose count 9.0/1.0 (8.0)
Epoch 015/020 | Loss 0.0010 | Win/lose count 11.0/3.0 (8.0)
Epoch 016/020 | Loss 0.0602 | Win/lose count 8.5

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
temp = np.arange(0.1,1,0.1)

for i in range (len(temp)):
    env = Environment(grid_size=size, max_time=T,temperature=temp[i])
    agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

    agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
    print('Temperature of {}'.format(temp[i]))
    print('  Test of the CNN')
    test(agent_cnn,env,epochs_test,prefix='cnn_test')
    print('  Test of the FC')
    test(agent_fc,env,epochs_test,prefix='fc_test')

Temperature of 0.1
  Test of the CNN
Win/lose count 5.0/4.0. Average score (1.0)
Win/lose count 2.5/0. Average score (1.75)
Win/lose count 1.5/0. Average score (1.6666666666666667)
Win/lose count 3.5/3.0. Average score (1.375)
Win/lose count 1.5/0. Average score (1.4)
Win/lose count 2.0/0. Average score (1.5)
Win/lose count 2.5/2.0. Average score (1.3571428571428572)
Win/lose count 0.5/1.0. Average score (1.125)
Win/lose count 2.0/1.0. Average score (1.1111111111111112)
Win/lose count 2.5/0. Average score (1.25)
Win/lose count 1.0/0. Average score (1.2272727272727273)
Win/lose count 1.5/1.0. Average score (1.1666666666666667)
Win/lose count 0.5/1.0. Average score (1.0384615384615385)
Win/lose count 0.5/0. Average score (1.0)
Win/lose count 0.5/0. Average score (0.9666666666666667)
Win/lose count 1.0/3.0. Average score (0.78125)
Win/lose count 1.5/1.0. Average score (0.7647058823529411)
Win/lose count 1.5/0. Average score (0.8055555555555556)
Win/lose count 1.0/1.0. Average score (0.763

In [0]:
HTML(display_videos('cnn_test10.mp4'))

In [0]:
HTML(display_videos('fc_test10.mp4'))

The CNN architecture performs globally better than the fully connected (FC) network. The final score with the CNN policy increases with the temperature, unlike the final score with the FC policy that remains quite unstable.

With both though, the agent seems to get stuck in a group of cells and not explore sufficiently. This will not decrease the score, however the agent is losing time circling between the same couple of cells. This phenomenon is intensified when the cat cannot see a positive reward from where it is, i.e. when the temperature is low.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    base_epsilon = agent.epsilon

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        pos_malus = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            pos_malus += env.last_pos_malus
            reward_malus = reward + env.last_pos_malus

            if reward_malus > 0:
                win = win + reward_malus
            if reward_malus < 0:
                lose = lose -reward_malus

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring, self).__init__(grid_size=grid_size, max_time=max_time, temperature=temperature)
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.last_pos_malus = 0  # remember the previous position malus

    def reset(self):
        state = super(EnvironmentExploring, self).reset() # reset env

        self.malus_position = np.zeros((self.grid_size, self.grid_size)) # reset malus
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),self.board.reshape(self.grid_size, self.grid_size,1),self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        # like in parent method, truncate state to a (5,5) window around the agent
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        
        return state

    def act(self, action):
        state, reward, game_over = super(EnvironmentExploring, self).act(action)
        
        if train:
            reward -= self.malus_position[self.x, self.y] # penalize if cell has been visited

        self.last_pos_malus = self.malus_position[self.x, self.y]  # remember the last position malus
        self.malus_position[self.x, self.y] = 0.25  # cell has been visited
        malus_position =  self.malus_position[self.x-2:self.x+3, self.y-2:self.y+3, np.newaxis] # restricting to seen cells
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),self.board.reshape(self.grid_size, self.grid_size,1),self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        # like in parent method, truncate state to a (5,5) window around the agent
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

In [0]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=1e-3, epsilon=0.1, memory_size=2000, batch_size=32, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/020 | Loss 0.0112 | Win/lose count 2.0/5.0 (-3.0)
Epoch 001/020 | Loss 0.0109 | Win/lose count 12.5/13.0 (-0.5)
Epoch 002/020 | Loss 0.0065 | Win/lose count 8.0/4.0 (4.0)
Epoch 003/020 | Loss 0.0214 | Win/lose count 19.5/2.0 (17.5)
Epoch 004/020 | Loss 0.0075 | Win/lose count 18.0/4.0 (14.0)
Epoch 005/020 | Loss 0.0503 | Win/lose count 25.0/5.0 (20.0)
Epoch 006/020 | Loss 0.0103 | Win/lose count 18.5/3.0 (15.5)
Epoch 007/020 | Loss 0.0088 | Win/lose count 19.0/6.0 (13.0)
Epoch 008/020 | Loss 0.0115 | Win/lose count 23.5/6.0 (17.5)
Epoch 009/020 | Loss 0.0137 | Win/lose count 20.5/4.0 (16.5)
Epoch 010/020 | Loss 0.0094 | Win/lose count 20.5/7.0 (13.5)
Epoch 011/020 | Loss 0.0077 | Win/lose count 23.0/6.0 (17.0)
Epoch 012/020 | Loss 0.0219 | Win/lose count 25.0/0 (25.0)
Epoch 013/020 | Loss 0.0114 | Win/lose count 18.5/5.0 (13.5)
Epoch 014/020 | Loss 0.0075 | Win/lose count 18.0/6.0 (12.0)
Epoch 015/020 | Loss 0.0103 | Win/lose count 24.5/2.0 (22.5)
Epoch 016/020 | Loss 0.0106 

In [0]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 18.5/35.75. Average score (-17.25)
Win/lose count 20.5/32.5. Average score (-14.625)
Win/lose count 20.0/38.75. Average score (-16.0)
Win/lose count 16.0/34.0. Average score (-16.5)
Win/lose count 7.0/44.0. Average score (-20.6)
Win/lose count 20.5/28.5. Average score (-18.5)
Win/lose count 21.5/32.25. Average score (-17.392857142857142)
Win/lose count 20.0/34.25. Average score (-17.0)
Win/lose count 24.0/29.5. Average score (-15.722222222222221)
Win/lose count 21.5/33.25. Average score (-15.325)
Win/lose count 18.0/34.5. Average score (-15.431818181818182)
Win/lose count 15.0/36.0. Average score (-15.895833333333334)
Win/lose count 15.5/37.0. Average score (-16.326923076923077)
Win/lose count 20.0/31.5. Average score (-15.982142857142858)
Win/lose count 10.0/45.25. Average score (-17.266666666666666)
Win/lose count 25.5/28.0. Average score (-16.34375)
Win/lose count 20.0/32.5. Average score (-16.11764705882353)
Win/lose count 21.0/33.0. Average score (-15.88888888888889

Increasing the position malus from 0.1 to 0.25 improved the results a lot because an exploration penalty encourages the agent to visit unseen cells.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***